In [0]:
import pyspark.sql.functions as F

india_df = spark.table('hive_metastore.azure_sql_db_dbo.crimes_on_women')
india_df.display()

column names are not clear, luckily the dataset includes a data description table
|      |   Column Names  |   Explanation                       |
|------|-----------------|-------------------------------------|
|   0  |   State         |   State                             |
|   1  |   Year          |   Year                              |
|   2  |   Rape          |   No. of Rape cases                 |
|   3  |   K&A           |   Kidnap And Assault                |
|   4  |   DD            |   Dowry Deaths                      |
|   5  |   AoW           |   Assault against women             |
|   6  |   AoM           |   Assault against modesty of women  |
|   7  |   DV            |   Domestic violence                 |
|   8  |   WT            |   Women Trafficking                 |

In [0]:
india_df_renamed = india_df.where(
    F.col('_fivetran_deleted') == False,
).withColumn(
    'country', F.lit('india'),
).select(
    F.col('country'),
    F.col('Year').alias('year'),
    F.col('State').alias('state'),
    F.col('Rape').alias('rape'),
    F.col('ka').alias('kidnap_assault'),
    F.col('dd').alias('dowry_deaths'),
    F.col('dv').alias('domestic_violence'),
    F.col('aow').alias('assault'),
    F.col('aom').alias('assault_on_modesty'),
)
india_df_renamed.display()

now we melt the table

In [0]:
india_df_melted = india_df_renamed.melt(
    ids=['country', 'year', 'state'],
    values=None,
    valueColumnName='total_cases',
    variableColumnName='type_of_violence',
)
india_df_melted.display()

sum all violence cases and yearly total

In [0]:
india_df_any_violence = india_df_melted.groupBy('year', 'country', 'state', 'type_of_violence').agg(
    F.sum('total_cases').alias('total_cases')
).withColumn(
    'type_of_violence', F.lit('any')
).select(
    F.col('year'),
    F.col('country'),
    F.col('state'),
    F.col('type_of_violence'),
    F.col('total_cases')
)
india_df_any_violence = india_df_melted.union(india_df_any_violence)
india_df_total = india_df_any_violence.groupBy('year', 'country', 'type_of_violence').agg(
    F.sum('total_cases').alias('total_cases')
).withColumn(
    'state', F.lit('all')
).select(
    F.col('year'),
    F.col('country'),
    F.col('state'),
    F.col('type_of_violence'),
    F.col('total_cases')
)
india_df_total = india_df_total.union(india_df_any_violence)
india_df_total.display()

now we need to delete the '_' character form the year